In [ ]:
import os
os.chdir("..")

In [15]:
from unitcellengine.design import UnitcellDesign, UnitcellDesigns
import numpy as np
import pyvista as pv
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from pathlib import Path

# Set numpy precision to allow for better printout of homogenized matrices
np.set_printoptions(precision=4)

# Single point design

Create a single point design of a unit aspect ratio Octet truss lattice and investigate its properties.
Note that, in the default setup, the assumed material has a modulus of elasticity of 1 and Poisson's ratio of 0.3.

In [ ]:
# Define unitcell
design = UnitcellDesign('Octet', 1, 1, 1, thickness=0.1)

# The default behavior is to reuse existing date. Set to False to force regeneration.
reuse = False

# Generate geometry (which calculated properties like relative density and relative surface area)
design.generateGeometry(reuse=reuse)

# Generate mesh
design.generateMesh(reuse=reuse)

# Calculate homogenized elastic properties
design.homogenizationElastic.run(reuse)

# Post process homogenization results
design.homogenizationElastic.process()

# Calculate homogenized conductance properties
design.homogenizationConductance.run(reuse)

# Post process conductance results
design.homogenizationConductance.process()

# Print the homogenizated stiffness matrix
print(design.homogenizationElastic)
print(design.homogenizationConductance)

## Visualize geometry

The scalar colors corresponds to the underlying signed-distance field, which defines the distance of a point to the closed point on the geometry surface (thus, a value of 0 corresponds to the surface of the geometry).

Note: the python package "trame", along with the trame-jupyter-extension jupyter extension, 
are required for display of an interactive plot. See https://tutorial.pyvista.org/tutorial/00_jupyter/index.html
for more details

In [ ]:
geometry = design.geometry.visualizeVTK()
geometry.plot()

In [ ]:
design = UnitcellDesign('Simple cubic', 1, 1, 1)
geometry = design.geometry.visualizeVTK()
mesh = geometry.extract_geometry().triangulate()
mesh.flip_normals()
mesh.plot()

This geometry can be exported to an STL (or other surface geometry form like obj or ply).

In [ ]:
geometry = design.geometry.visualizeVTK()
mesh = geometry.extract_geometry().triangulate()
mesh.flip_normals()
mesh.save("geometry.stl")

Additionally, the resolution of the generated geometry can be increased at the cost of increased generation time.
This is done by specifying the discretization size of the geometry through its "elementSize" property.
This property is defined relative to the thickness parameter of the unitcell.
So, a value of 1/4 corresponds to a discretization that can fit 4 discrete elements across the thickness of the unitcell beams/faces.
Thus, as elementSize decreases, the resolution increases.

In [ ]:
design.geometry.elementSize = 0.25
geometry = design.geometry.visualizeVTK()
mesh = geometry.extract_geometry().triangulate()
mesh.flip_normals()
mesh.plot()

## Fillets in graph unitcells

Graph unitcells are defined by sets of beams and faces.
At the intersection of these beams and faces, it is possible to specify a fillet radius.
This is done by specifying the "radius" parameter in the UnitcellDesign, which defines the fillet radius relative to the beam/face thickness.
For example, if the beam/face thickness is 0.3 mm and radius=0.5, then the corresponding fillet radius of 0.5*0.3mm=0.15 mm.
The below example shows the same "Simple Cubic" unitcell with varying fillet radii.

In [ ]:
meshes = []
pl = pv.Plotter(shape=(1, 3))
for i, radius in enumerate([0., 0.5, 1.]):
    design = UnitcellDesign('Simple cubic', 1, 1, 1, radius=radius)
    geometry = design.geometry.visualizeVTK()
    mesh = geometry.extract_geometry().triangulate()
    mesh.flip_normals()
    pl.subplot(0, i)
    pl.add_mesh(mesh)
    pl.add_text(f"Radius: {radius}")

pl.link_views()
pl.view_isometric()
pl.show()



## Visualize anistropy of elastic properties

For more details regaring the below visualizations, see Nordmann, J., Aßmus, M., & Altenbach, H. (2018). *Visualising elastic anisotropy: theoretical background and computational implementation*. Continuum Mechanics and Thermodynamics, 30, 689-708.

In [22]:
# Define unitcell
design = UnitcellDesign('Octet', 1, 1, 1, thickness=0.1)

# Reuse data calculate earlier in the notebook
reuse = True

Plotting the directional dependence of the lattice stiffness provides insight into the anisotropy of the structure.
If the structure was isotropic, the corresponding plot would be a sphere as the stiffness is the same in all loading directions.
As the plot deviates from the sphere, the anisotropy of the structure increases.

In [ ]:
design.homogenizationElastic.plotE()

Visualization and interpretation of the other structural properties, like shear modulus (plotG) and Poisson's ratio (plotNu), is more challenging because of their depending on both loading direction and plane of reference. 
Below is an example of the shear modulus properties.


In [ ]:
design.homogenizationElastic.plotG()

# Export design to a centeralized database

Export to a centeralized HDF5 database

In [ ]:
design.export()

The centralized database is located in the root database folder and can be found as follows:

In [ ]:
design.databaseFilename

# Design groups

Generate the properties for a set of lattices and manage their properties as a group.

In [ ]:
cases = [
    dict(length=1, width=1, height=1, thickness=0.2),
    dict(length=2, width=1, height=1, thickness=0.2),
    dict(length=3, width=1, height=1, thickness=0.2),
    dict(length=4, width=1, height=1, thickness=0.2),
]

# The default behavior is to reuse existing date. Set to False to force regeneration.
reuse = False

for case in tqdm(cases):
    # Define unitcell
    design = UnitcellDesign('Gyroid', **case)
    
    # Generate geometry (which calculated properties like relative density and relative surface area)
    design.generateGeometry(reuse=reuse)
    
    # Generate mesh
    design.generateMesh(reuse=reuse)
    
    # Calculate homogenized elastic properties
    design.homogenizationElastic.run(reuse=reuse, solver='iterative')
    
    # Post process homogenization results
    design.homogenizationElastic.process(reuse=False)

Load the data for all of the Octet truss designs

In [ ]:
designs = UnitcellDesigns("Gyroid", form="walledtpms")

Get various properties from the set of unitcell designs

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=designs.lengths, y=designs.relativeDensities))
fig.update_xaxes(title="Unitcell length")
fig.update_yaxes(title="Relative density")

Calculate the normalized elastic stiffness in the 1 direction (i.e., length-wise direction)

In [ ]:
E11s = [d.homogenizationElastic.Ei([1, 0, 0]) for d in designs.designs]
fig = go.Figure()
fig.add_trace(go.Scatter(x=designs.relativeDensities, y=E11s))
fig.update_yaxes(title="E<sub>11</sub>")
fig.update_xaxes(title="Relative density")

# Create a custom unitcell

UnitcellEngine currently allows for the creation of custom "graph" style unitcells. 
To do so, a template for the unitcell geometry needs to be created.
This template is a python dictionary with the keys, "node", "beam", and "face":
- "node" is an Nx3 numpy array defining the coordinates of N vertices in the unitcell geometry.
- "beam" is [] or None if the unitcell has no beams; otherwise, it is an Mx2 numpy array defining the vertex connectivity of M beams by specifying the "node" integer indeces for the starting and ending points (noting that 0 indexing is used)
- "face" is [] or None if the unitcell has no faces; otherwise, it is an PxQ numpy array defining the vertex connectivity of P faces by specifying the "node" integer indeces for the Q points in the face ordered in a clock-wise direction (i.e., order matters). Faces can be triangles (Q=3) or quadralaterals (Q=4). If it is desirable to mix triangles and quadralaterals, set Q=4 and specify an index of -1 in the last column for any triangle references. 
T


Here, we'll replicate the creation of the simple cubic unitcell from scratch.

In [ ]:
template = {
    "node": np.array(
        [
            [-0.5, -0.5, -0.5],
            [0.5, -0.5, -0.5],
            [-0.5, 0.5, -0.5],
            [0.5, 0.5, -0.5],
            [-0.5, -0.5, 0.5],
            [0.5, -0.5, 0.5],
            [-0.5, 0.5, 0.5],
            [0.5, 0.5, 0.5],
        ]
    ),
    "beam": np.array(
        [
            [0, 1],
            [0, 2],
            [2, 3],
            [1, 3],
            [4, 5],
            [4, 6],
            [6, 7],
            [5, 7],
            [0, 4],
            [1, 5],
            [2, 6],
            [3, 7],
        ]
    ),
    "face": {},
}
custom = dict(name="Custom Simple Cubic", definition=template)
design = UnitcellDesign(
    custom,
    1,
    1,
    3,
    thickness=0.3,
    radius=0.1,
    # directory=Path(__file__).parent / Path("tests"),
    elementSize=0.5,
    form="graph",
)
design.geometry.run(reuse=False, export=True)

In [ ]:
geometry = design.geometry.visualizeVTK()
mesh = geometry.extract_geometry().triangulate()
mesh.flip_normals()
mesh.plot()